In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# стили
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sns.set_theme(context='talk', style='whitegrid', palette='deep')
plt.rcParams['figure.figsize'] = 10, 6
plt.rcParams['font.size'] = 20
plt.rcParams['axes.labelsize'] = 25
plt.rcParams['figure.titlesize'] = 28
plt.rcParams['axes.titlesize'] = 24
plt.rcParams['savefig.format'] = 'pdf'
plt.rcParams['figure.autolayout'] = 'true'
plt.rcParams['figure.frameon'] = 'false'
plt.rcParams['axes.spines.left'] = 'false'
plt.rcParams['axes.spines.right'] = 'false'
plt.rcParams['axes.spines.top'] = 'false'
plt.rcParams['legend.fancybox'] = 'false'
plt.rcParams['axes.spines.bottom'] = 'false'

plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'
# для графиков, где надо много цветов
# sns.set_palette(sns.color_palette('deep'))
# а по дефолту 
sns.set_palette(sns.color_palette("Blues", n_colors=5)[1:])
pd.set_option('display.max_columns', 60)

In [10]:
from dotenv import load_dotenv
import os
load_dotenv()
path = os.getenv("DATA_PATH")

In [18]:
df = pd.read_csv(path, sep=';', low_memory=False)
df.head()

,new_id,Месяц,Трафик,Средний чек,"Дата открытия, категориальный","Торговая площадь, категориальный",Населенный пункт,Регион,Численность населения,Количество домохозяйств,"Трафик пеший, в час","Трафик авто, в час","Маркетплейсы, доставки, постаматы (100 м)",Медицинские уч. и аптеки (300 м),Школы (300 м),Остановки (300 м),Продуктовые магазины (500 м),Пятерочки (500 м)
0,0,10,59662,"823,0603898",Средний по возрасту,Средний,Кавказская ст-ца,Краснодарский край,10177,608,"76,9","200,333333",0,6,0,0,2,0
1,0,5,56674,"859,3619753",Средний по возрасту,Средний,Кавказская ст-ца,Краснодарский край,10177,608,"76,9","200,333333",0,6,0,0,2,0
2,0,1,51488,"763,9377659",Средний по возрасту,Средний,Кавказская ст-ца,Краснодарский край,10177,608,"76,9","200,333333",0,6,0,0,2,0
3,0,6,56693,"836,3623093",Средний по возрасту,Средний,Кавказская ст-ца,Краснодарский край,10177,608,"76,9","200,333333",0,6,0,0,2,0
4,0,7,58128,"845,2577093",Средний по возрасту,Средний,Кавказская ст-ца,Краснодарский край,10177,608,"76,9","200,333333",0,6,0,0,2,0


In [19]:
# Приводим все целевые поля к числам
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='ignore')

# Считаем число пустых значений
display(df[list(df.columns)].isna().sum())


C:\Users\Daria\AppData\Local\Temp\ipykernel_16700\3572225280.py:3: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


new_id                                       0
Месяц                                        0
Трафик                                       0
Средний чек                                  0
Дата открытия, категориальный                0
Торговая площадь, категориальный             0
Населенный пункт                             0
Регион                                       0
Численность населения                        0
Количество домохозяйств                      0
Трафик пеший, в час                          0
Трафик авто, в час                           0
Маркетплейсы, доставки, постаматы (100 м)    0
Медицинские уч. и аптеки (300 м)             0
Школы (300 м)                                0
Остановки (300 м)                            0
Продуктовые магазины (500 м)                 0
Пятерочки (500 м)                            0
dtype: int64

In [ ]:
df.info() # из-за запятых вместо точек не все распознано как числа

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256723 entries, 0 to 256722
Data columns (total 18 columns):
 #   Column                                     Non-Null Count   Dtype 
---  ------                                     --------------   ----- 
 0   new_id                                     256723 non-null  int64 
 1   Месяц                                      256723 non-null  int64 
 2   Трафик                                     256723 non-null  int64 
 3   Средний чек                                256723 non-null  object
 4   Дата открытия, категориальный              256723 non-null  object
 5   Торговая площадь, категориальный           256723 non-null  object
 6   Населенный пункт                           256723 non-null  object
 7   Регион                                     256723 non-null  object
 8   Численность населения                      256723 non-null  int64 
 9   Количество домохозяйств                    256723 non-null  int64 
 10  Трафик пеший, в час 

In [ ]:
cols = {
    'new_id': 'new_id',
    'month': 'Месяц',
    "open_date": "Дата открытия, категориальный",
    "area": "Торговая площадь, категориальный",
    "locality": "Населенный пункт",
    "region": "Регион",
    'traffic': 'Трафик',                     # среднее количество посетителей в месяц
    'avg_check': 'Средний чек',              # средний чек
    'population': 'Численность населения',   # население
    'households': 'Количество домохозяйств', # домохозяйств в радиусе 500м
    'ped_traffic': 'Трафик пеший, в час',    # пешеходный трафик
    'auto_traffic': 'Трафик авто, в час',    # авто трафик
    'marketplaces': 'Маркетплейсы, доставки, постаматы (100 м)',
    'medical': 'Медицинские уч. и аптеки (300 м)',
    'schools': 'Школы (300 м)',
    'stops': 'Остановки (300 м)',
    'grocery_competitors': 'Продуктовые магазины (500 м)',
    'pyaterochki_near': 'Пятёрочки (500 м)'
}

,new_id,Месяц,Трафик,Средний чек,"Дата открытия, категориальный","Торговая площадь, категориальный",Населенный пункт,Регион,Численность населения,Количество домохозяйств,"Трафик пеший, в час","Трафик авто, в час","Маркетплейсы, доставки, постаматы (100 м)",Медицинские уч. и аптеки (300 м),Школы (300 м),Остановки (300 м),Продуктовые магазины (500 м),Пятерочки (500 м)
0,0,10,59662,"823,0603898",Средний по возрасту,Средний,Кавказская ст-ца,Краснодарский край,10177,608,"76,9","200,333333",0,6,0,0,2,0
1,0,5,56674,"859,3619753",Средний по возрасту,Средний,Кавказская ст-ца,Краснодарский край,10177,608,"76,9","200,333333",0,6,0,0,2,0
2,0,1,51488,"763,9377659",Средний по возрасту,Средний,Кавказская ст-ца,Краснодарский край,10177,608,"76,9","200,333333",0,6,0,0,2,0
3,0,6,56693,"836,3623093",Средний по возрасту,Средний,Кавказская ст-ца,Краснодарский край,10177,608,"76,9","200,333333",0,6,0,0,2,0
4,0,7,58128,"845,2577093",Средний по возрасту,Средний,Кавказская ст-ца,Краснодарский край,10177,608,"76,9","200,333333",0,6,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256718,21742,10,51676,"1167,101083",Новый,Средний,Октябрьский рп,Волгоградская обл,6071,262,"125,375","243,333333",1,1,0,0,1,0
256719,21742,11,51516,"1252,914118",Новый,Средний,Октябрьский рп,Волгоградская обл,6071,262,"125,375","243,333333",1,1,0,0,1,0
256720,21742,9,49593,"1130,823998",Новый,Средний,Октябрьский рп,Волгоградская обл,6071,262,"125,375","243,333333",1,1,0,0,1,0
256721,21742,12,52115,"1461,929305",Новый,Средний,Октябрьский рп,Волгоградская обл,6071,262,"125,375","243,333333",1,1,0,0,1,0
